In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Train data
train = pd.read_csv('train_data.csv')
train.head(n=10)

,title,data
0,__label__18-24,1917715178553519 866004230174021 5677300099090...
1,__label__55+,1549131271824144 544900528944370 1003942339965...
2,__label__18-24,708982919265866 1691719041155281
3,__label__55+,207189973202782 236035250274969 17434375003039...
4,__label__45-54,721926094625837 1868157670136246 1606679136294...
5,__label__18-24,426003864523918 107418812730138 16350066757891...
6,__label__25-34,390778261331214 372809096205922 12389533628014...
7,__label__45-54,130769934251370 833301643369441
8,__label__25-34,899640426733908 498710223497210 54414286240280...
9,__label__25-34,1513217328975937 881814218548682 3836561084061...


In [ ]:
# Test data
test = pd.read_csv('data_for_test.csv')
test.head(n=10)

,data
0,1516066978617491 203034046973744 1790864221223...
1,1802796346689443 1500653306612601 103169509694...
2,195820021235475 259202637839186 43681858976204...
3,2056365137910940 653191634844649 1747444955568...
4,313390485427979 464527593637994 19912760712863...
5,100802116791906
6,424163804628076 1287991051244372 1490894023997...
7,1416190268411395
8,261396520702360 415793858487646 35579238142041...
9,238001596744209 1507052469417587 1706808629341...


In [ ]:
#check that there are no missing values in either training set
print('The dataset has', train.isna().sum().sum(), 'missing values in train data.')
print('The dataset has', test.isna().sum().sum(), 'missing values in test data.')

# check if there are any duplicates
print('The dataset has', train.duplicated().sum(), 'duplicates in train data.')
print('The dataset has', test.duplicated().sum(), 'duplicates in test data.')

The dataset has 0 missing values in train data.
The dataset has 0 missing values in test data.
The dataset has 0 duplicates in train data.
The dataset has 24 duplicates in test data.


In [ ]:
# remove duplicates in train data
train.drop_duplicates(subset=['title', 'data'], inplace=True)

In [ ]:
# remove duplicates in test data
test.drop_duplicates(subset=['data'], inplace=True)

**DATA PREPROCESSING**

In [ ]:
X_train, y_train = train['data'], train['title']
X_test = test['data']



*   *Transforming text to a vector*
*Creating tf-idf feature


*   encode labels
*   compute tf-idf

https://www.kaggle.com/roccoli/multi-label-classification-with-sklearn





In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
# create a count vectorizer object
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(X_train)
# Transform documents to document-term matrix.
X_train_count = count_vect.transform(X_train)
X_test_count = count_vect.transform(X_test)

# preprocessing
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(y_train)
Y = multilabel_binarizer.transform(y_train)

tfidf_vect = TfidfVectorizer(analyzer='word', max_features=90000)
tfidf_vect.fit(X_train) # learn vocabulary and idf from training set
X_data_tfidf = tfidf_vect.transform(X_train)
# giả sử không có tập test trước đó 
X_test_tfidf = tfidf_vect.transform(X_test)


In [ ]:
y_train

0        __label__18-24
1          __label__55+
2        __label__18-24
3          __label__55+
4        __label__45-54
              ...      
32117    __label__35-44
32118    __label__25-34
32119    __label__25-34
32120    __label__35-44
32121    __label__25-34
Name: title, Length: 32003, dtype: object

**Training a classifier**

In [ ]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [ ]:
def train_model(classifier, X_train, y_train, X_test):           
    classifier.fit(X_train, y_train)
    train_predictions = classifier.predict(X_train)
    test_predictions = classifier.predict(X_test)        
    print("Training accuracy: ", accuracy_score(train_predictions, y_train))    

    return test_predictions

**Result 1**

In [ ]:
result = train_model(SGDClassifier(), X_data_tfidf, y_train, X_test_tfidf)

ValueError: ignored

In [ ]:
len(result)

13768

In [ ]:
import csv
file = './nhom10_AD_SOL1.txt'
with open(file, 'w') as f:
    for data in result:
        f.writelines(data + "\n")

**Result 2**

In [ ]:
result2 = train_model(MultinomialNB(), X_data_tfidf, y_train, X_test_tfidf)

Training accuracy:  0.7679280067493672


In [ ]:
len(result2)

13768

In [ ]:
file = './nhom10_AD_SOL2.txt'
with open(file, 'w') as f:
    for data in result2:
        f.writelines(data + "\n")

**Result 3**

In [ ]:
from sklearn.neighbors import NearestCentroid
result3 = train_model(NearestCentroid(), X_data_tfidf, y_train, X_test_tfidf)

Training accuracy:  0.7482111052088867


In [ ]:
file = './nhom10_AD_SOL3.txt'
with open(file, 'w') as f:
    for data in result3:
        f.writelines(data + "\n")

**Result 4**

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
result4 = train_model(OneVsRestClassifier(SGDClassifier()), X_data_tfidf, y_train, X_test_tfidf)

Training accuracy:  0.8802612255101084


In [ ]:
file = './nhom10_AD_SOL4.txt'
with open(file, 'w') as f:
    for data in result4:
        f.writelines(data + "\n")

**Result 5**

In [ ]:
result5 = train_model(OneVsRestClassifier(MultinomialNB()), X_data_tfidf, y_train, X_test_tfidf)

Training accuracy:  0.7872386963722151


In [ ]:
file = './nhom10_AD_SOL5.txt'
with open(file, 'w') as f:
    for data in result5:
        f.writelines(data + "\n")